In [1]:
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import os, json, open_clip, torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F

import os
import json
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from natsort import natsorted
import pandas as pd
from tqdm import tqdm
import open_clip
from datetime import datetime

In [22]:
# 0. Settings
device = 'cuda:1'
model_name = 'ViT-L-14'
pretrained = 'openai'
root = './'
dataset_name = 'Scene'

# 1. Load CLIP model
model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
tokenizer = open_clip.get_tokenizer(model_name)

In [23]:
# 2. Load test dataset with data augmentation
ds = ImageFolder(os.path.join(root, dataset_name), transform=preprocess)
ds.samples = natsorted(ds.samples)
dl = DataLoader(ds, shuffle=False, batch_size=32, num_workers=2)

In [24]:
model.logit_scale

Parameter containing:
tensor(4.6052, requires_grad=True)

In [25]:
# 3. Load class name list
with open(os.path.join(root, 'classes.json'), 'r') as j:
    class_names = json.loads(j.read())

# Enhanced text prompts
prompts = [f"A photo of a {class_name}" for class_name in class_names]

# 4. Perform zero-shot classification
zero_shot_top1 = 0
submission = dict({'id_idx': list(range(8100)), 'label': []})

with torch.no_grad(), torch.cuda.amp.autocast():
    # Text to CUDA device (GPU 1)
    text = tokenizer(prompts)
    text_features = model.encode_text(text).to('cuda:1')
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    model = model.to(device)  # Move model to the specified GPU

    for x, y in tqdm(dl):
        x = x.cuda(device).float()  # Move images to GPU and ensure float32
        image_features = model.encode_image(x).to('cuda:1').float()  # Convert to float32 explicitly
        image_features /= image_features.norm(dim=-1, keepdim=True)
        
        # 32-bit computations with text and image features
        zero_shot_probs = F.softmax((50 * image_features @ text_features.T).float())  # Ensure float32 operation

        rst = zero_shot_probs.tolist()  # Convert to list for submission

        for i in rst:
            print(i)
        break

        submission['label'] += rst

# 5. Save prediction as submission.csv file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'1-1_soft_submission_{current_time}.csv'
pd.DataFrame(submission).to_csv(os.path.join(root, file_name), index=False)

  0%|                                                   | 0/254 [00:00<?, ?it/s]/tmp/ipykernel_3217542/4224370164.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  zero_shot_probs = F.softmax((50 * image_features @ text_features.T).float())  # Ensure float32 operation
  0%|                                                   | 0/254 [00:00<?, ?it/s]

[0.25929996371269226, 0.07908201962709427, 0.04070793837308884, 0.06454486399888992, 0.24358977377414703, 0.31277546286582947]
[0.6517913937568665, 0.0370599590241909, 0.030723784118890762, 0.11150836199522018, 0.07428057491779327, 0.09463591128587723]
[0.026973672211170197, 0.6638035774230957, 0.05755113065242767, 0.11625649034976959, 0.08844327181577682, 0.04697186127305031]
[0.6621710658073425, 0.03482067957520485, 0.034014057368040085, 0.06063661351799965, 0.07487622648477554, 0.13348129391670227]
[0.05400538071990013, 0.06167608126997948, 0.22737114131450653, 0.2829679548740387, 0.352159321308136, 0.02182014472782612]
[0.16946831345558167, 0.3396713137626648, 0.03993900492787361, 0.07698444277048111, 0.1935388594865799, 0.18039806187152863]
[0.7321286201477051, 0.021261408925056458, 0.008229086175560951, 0.028947606682777405, 0.0286103542894125, 0.18082301318645477]
[0.06614239513874054, 0.06824197620153427, 0.192890465259552, 0.5367647409439087, 0.10164612531661987, 0.03431422263

ValueError: All arrays must be of the same length